# Algorithme d'estimation de phase
Francis Blais (2151696) et Vincent Girouard (2152383)

12 novembre 2023
## Objectif de l'algorithme
L'algorithme d'estimation de phase est un algorithme assez particulier dans le sens que le problème qu'il essaie de résoudre est purement quantique***. On considère un opérateur unitaire inconnu $U$ et un état d'entrée $\ket{\psi}$ de $n$ qubits. Il nous est assuré que $\ket{\psi}$ est un état propre de $U$ tel que

$$ U\ket{\psi} = \lambda\ket{\psi}  $$

où $\lambda$ est la valeur propre associée à l'état $\ket{\psi}$. Comme $U$ est unitaire, $|\lambda|^2=1$ ce qui permet de simplifier le problème en décrivant complètement la valeur propre par une phase $\theta\in[0,1[$ comme suit : $\lambda = e^{i2\pi\theta}$. Comme son nom l'indique, l'algorithme d'estimation de phase a pour objectif d'approximer selon une précision voulue la valeur de $\theta$. On parle ici d'approximation, car la précision à laquelle il sera possible de connaitre $\theta$ dépendra du nombre de quibit de référence choisi. Cet algorithme diffère un peu d'autres algorithmes quantiques dans le sens qu'ici, l'entrée comprend un état quantique $\ket{\psi}$. De ce fait, cet algorithme est souvent utilisé en pair avec d'autres algorithmes plus complexes. C'est noramment une partie très importante de l'algorithme de Shor permettant de factoriser des entiers. 

## Description de l'algorithme

## Principe de fonctionnement

## Implémentation

In [3]:
import numpy as np
from numpy import pi
from qutip import Qobj, about
from qutip.qip import *
from qutip.qip.circuit import QubitCircuit
from qutip.qip.operations import *
# (Gate, berkeley, cnot, cphase, csign, fredkin,
#                                   gate_sequence_product, globalphase, iswap,
#                                   molmer_sorensen, phasegate, qrot, rx, ry, rz,
#                                   snot, sqrtiswap, sqrtnot, sqrtswap, swap,
#                                   swapalpha, toffoli)

%matplotlib inline

In [5]:
q = QubitCircuit(2, reverse_states=False)
q.add_gate("CSIGN", controls=[0], targets=[1])
q.png

RuntimeError: Could not find system 'pdflatex'.